In [54]:
import pandas as pd 

In [55]:
ds = pd.read_csv("antwerp_ds_modified.csv", index_col=0) 

In [56]:
ds

,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,BS 64,BS 65,BS 66,BS 67,BS 68,RX Time,SF,HDOP,Latitude,Longitude
0,-200,-200,-200,-200,-200,-200,-200,-200,-114,-200,...,-200,-200,-200,-200,-200,2017-12-28 16:14:07.247000+00:00,12,1.02,51.216736,4.412130
1,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-11-28 12:01:02.380000+00:00,7,0.57,51.232037,4.405185
2,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-12-14 09:05:22.832000+00:00,9,1.00,51.202763,4.390477
3,-200,-100,-200,-200,-200,-200,-200,-200,-200,-113,...,-200,-200,-200,-200,-200,2018-01-17 09:11:49.222000+00:00,12,0.62,51.194584,4.417887
4,-200,-106,-107,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-10 14:47:01.379000+00:00,12,1.00,51.207203,4.403470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123523,-200,-103,-106,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-04 10:57:15.856000+00:00,12,0.61,51.201839,4.414310
123524,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2018-01-03 23:17:34.799000+00:00,8,1.10,51.233677,4.416892
123525,-200,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,-200,2017-12-15 10:13:56.383000+00:00,12,0.82,51.215370,4.412519
123526,-200,-97,-200,-200,-200,-200,-200,-200,-200,-92,...,-200,-200,-200,-200,-200,2018-01-12 23:33:09.229000+00:00,12,0.61,51.194252,4.418354


In [57]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

In [58]:
def get_weather_info(lat, long, time:str):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": lat,
        "longitude": long,
        "elevation": 5.0,
        "start_hour": time,
        "end_hour": time,
        "hourly": ["temperature_2m", 
                "relative_humidity_2m",
                "dew_point_2m", 
                "apparent_temperature", 
                "precipitation", 
                "rain", 
                "snowfall", 
                "snow_depth", 
                "weather_code", 
                "pressure_msl", 
                "surface_pressure", 
                "cloud_cover", 
                "cloud_cover_low", 
                "cloud_cover_mid", 
                "cloud_cover_high", 
                "et0_fao_evapotranspiration", 
                "vapour_pressure_deficit", 
                "wind_speed_10m", 
                "wind_speed_100m", 
                "wind_direction_10m", 
                "wind_direction_100m", 
                "wind_gusts_10m", 
                "shortwave_radiation", 
                "direct_radiation",
                "diffuse_radiation", 
                "direct_normal_irradiance", 
                "global_tilted_irradiance", 
                "terrestrial_radiation", 
                "shortwave_radiation_instant",
                "direct_radiation_instant", 
                "diffuse_radiation_instant", 
                "direct_normal_irradiance_instant", 
                "global_tilted_irradiance_instant", 
                "terrestrial_radiation_instant"]
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
    hourly_rain = hourly.Variables(5).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
    hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
    hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
    hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
    hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
    hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
    hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
    hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
    hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
    hourly_shortwave_radiation = hourly.Variables(22).ValuesAsNumpy()
    hourly_direct_radiation = hourly.Variables(23).ValuesAsNumpy()
    hourly_diffuse_radiation = hourly.Variables(24).ValuesAsNumpy()
    hourly_direct_normal_irradiance = hourly.Variables(25).ValuesAsNumpy()
    hourly_global_tilted_irradiance = hourly.Variables(26).ValuesAsNumpy()
    hourly_terrestrial_radiation = hourly.Variables(27).ValuesAsNumpy()
    hourly_shortwave_radiation_instant = hourly.Variables(28).ValuesAsNumpy()
    hourly_direct_radiation_instant = hourly.Variables(29).ValuesAsNumpy()
    hourly_diffuse_radiation_instant = hourly.Variables(30).ValuesAsNumpy()
    hourly_direct_normal_irradiance_instant = hourly.Variables(31).ValuesAsNumpy()
    hourly_global_tilted_irradiance_instant = hourly.Variables(32).ValuesAsNumpy()
    hourly_terrestrial_radiation_instant = hourly.Variables(33).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["rain"] = hourly_rain
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["snow_depth"] = hourly_snow_depth
    hourly_data["weather_code"] = hourly_weather_code
    hourly_data["pressure_msl"] = hourly_pressure_msl
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
    hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
    hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
    hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
    hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
    hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
    hourly_data["direct_radiation"] = hourly_direct_radiation
    hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
    hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
    hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
    hourly_data["terrestrial_radiation"] = hourly_terrestrial_radiation
    hourly_data["shortwave_radiation_instant"] = hourly_shortwave_radiation_instant
    hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant
    hourly_data["diffuse_radiation_instant"] = hourly_diffuse_radiation_instant
    hourly_data["direct_normal_irradiance_instant"] = hourly_direct_normal_irradiance_instant
    hourly_data["global_tilted_irradiance_instant"] = hourly_global_tilted_irradiance_instant
    hourly_data["terrestrial_radiation_instant"] = hourly_terrestrial_radiation_instant

    hourly_dataframe = pd.DataFrame(data = hourly_data)
    return hourly_dataframe

In [72]:
get_weather_info(ds["Latitude"][0], ds["Longitude"][0], "2017-12-28T16:14")

Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
0,2017-12-28 16:14:00+00:00,3.2455,74.766975,-0.8045,-1.471424,0.0,0.0,0.0,0.0,1.0,...,5.0,51.51376,7.999999,21.081083,0.0,0.0,0.0,0.0,0.0,0.0


In [67]:
# Convert the RX Time values to datetime format ISO8601 and store in a separate column upto minutes precision
# Expected format "2017-12-28T16:14"
ds["Weather param sense time"] = pd.to_datetime(ds["RX Time"], format='ISO8601').dt.strftime('%Y-%m-%dT%H:%M')

In [68]:
f"{ds["Weather param sense time"][0]}"

'2017-12-28T16:14'

In [117]:
def get_weather_info_mod(ds_row):
    # Setup the Open-Meteo API client with cache and retry on error
    cache_session = requests_cache.CachedSession('.cache', expire_after = -1)
    retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
    openmeteo = openmeteo_requests.Client(session = retry_session)

    # Make sure all required weather variables are listed here
    # The order of variables in hourly or daily is important to assign them correctly below
    url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": ds_row["Latitude"],
        "longitude": ds_row["Longitude"],
        "elevation": 5.0,
        "start_hour": f"{ds_row["Weather param sense time"]}",
        "end_hour": f"{ds_row["Weather param sense time"]}",
        "hourly": ["temperature_2m", 
                "relative_humidity_2m",
                "dew_point_2m", 
                "apparent_temperature", 
                "precipitation", 
                "rain", 
                "snowfall", 
                "snow_depth", 
                "weather_code", 
                "pressure_msl", 
                "surface_pressure", 
                "cloud_cover", 
                "cloud_cover_low", 
                "cloud_cover_mid", 
                "cloud_cover_high", 
                "et0_fao_evapotranspiration", 
                "vapour_pressure_deficit", 
                "wind_speed_10m", 
                "wind_speed_100m", 
                "wind_direction_10m", 
                "wind_direction_100m", 
                "wind_gusts_10m", 
                "shortwave_radiation", 
                "direct_radiation",
                "diffuse_radiation", 
                "direct_normal_irradiance", 
                "global_tilted_irradiance", 
                "terrestrial_radiation", 
                "shortwave_radiation_instant",
                "direct_radiation_instant", 
                "diffuse_radiation_instant", 
                "direct_normal_irradiance_instant", 
                "global_tilted_irradiance_instant", 
                "terrestrial_radiation_instant"]
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
    hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
    hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
    hourly_rain = hourly.Variables(5).ValuesAsNumpy()
    hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
    hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
    hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
    hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
    hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
    hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
    hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
    hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
    hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
    hourly_et0_fao_evapotranspiration = hourly.Variables(15).ValuesAsNumpy()
    hourly_vapour_pressure_deficit = hourly.Variables(16).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(17).ValuesAsNumpy()
    hourly_wind_speed_100m = hourly.Variables(18).ValuesAsNumpy()
    hourly_wind_direction_10m = hourly.Variables(19).ValuesAsNumpy()
    hourly_wind_direction_100m = hourly.Variables(20).ValuesAsNumpy()
    hourly_wind_gusts_10m = hourly.Variables(21).ValuesAsNumpy()
    hourly_shortwave_radiation = hourly.Variables(22).ValuesAsNumpy()
    hourly_direct_radiation = hourly.Variables(23).ValuesAsNumpy()
    hourly_diffuse_radiation = hourly.Variables(24).ValuesAsNumpy()
    hourly_direct_normal_irradiance = hourly.Variables(25).ValuesAsNumpy()
    hourly_global_tilted_irradiance = hourly.Variables(26).ValuesAsNumpy()
    hourly_terrestrial_radiation = hourly.Variables(27).ValuesAsNumpy()
    hourly_shortwave_radiation_instant = hourly.Variables(28).ValuesAsNumpy()
    hourly_direct_radiation_instant = hourly.Variables(29).ValuesAsNumpy()
    hourly_diffuse_radiation_instant = hourly.Variables(30).ValuesAsNumpy()
    hourly_direct_normal_irradiance_instant = hourly.Variables(31).ValuesAsNumpy()
    hourly_global_tilted_irradiance_instant = hourly.Variables(32).ValuesAsNumpy()
    hourly_terrestrial_radiation_instant = hourly.Variables(33).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["dew_point_2m"] = hourly_dew_point_2m
    hourly_data["apparent_temperature"] = hourly_apparent_temperature
    hourly_data["precipitation"] = hourly_precipitation
    hourly_data["rain"] = hourly_rain
    hourly_data["snowfall"] = hourly_snowfall
    hourly_data["snow_depth"] = hourly_snow_depth
    hourly_data["weather_code"] = hourly_weather_code
    hourly_data["pressure_msl"] = hourly_pressure_msl
    hourly_data["surface_pressure"] = hourly_surface_pressure
    hourly_data["cloud_cover"] = hourly_cloud_cover
    hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
    hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
    hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
    hourly_data["et0_fao_evapotranspiration"] = hourly_et0_fao_evapotranspiration
    hourly_data["vapour_pressure_deficit"] = hourly_vapour_pressure_deficit
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["wind_speed_100m"] = hourly_wind_speed_100m
    hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
    hourly_data["wind_direction_100m"] = hourly_wind_direction_100m
    hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation
    hourly_data["direct_radiation"] = hourly_direct_radiation
    hourly_data["diffuse_radiation"] = hourly_diffuse_radiation
    hourly_data["direct_normal_irradiance"] = hourly_direct_normal_irradiance
    hourly_data["global_tilted_irradiance"] = hourly_global_tilted_irradiance
    hourly_data["terrestrial_radiation"] = hourly_terrestrial_radiation
    hourly_data["shortwave_radiation_instant"] = hourly_shortwave_radiation_instant
    hourly_data["direct_radiation_instant"] = hourly_direct_radiation_instant
    hourly_data["diffuse_radiation_instant"] = hourly_diffuse_radiation_instant
    hourly_data["direct_normal_irradiance_instant"] = hourly_direct_normal_irradiance_instant
    hourly_data["global_tilted_irradiance_instant"] = hourly_global_tilted_irradiance_instant
    hourly_data["terrestrial_radiation_instant"] = hourly_terrestrial_radiation_instant

    hourly_dataframe = pd.DataFrame(data = hourly_data, index=[ds_row.name])  # Here we set index to the value of the original row index
    # this will help us concatinating the weather data to the main dataset
    # the index are only identifier of each sample and their corresponding weather data 
    return hourly_dataframe

In [118]:
# Sample of n are tested for getting weather data
sample_ds = ds.sample(n=10)
rdf = pd.concat([get_weather_info_mod(sample_ds.iloc[i, :]) for i in  range(len(sample_ds))], axis=0)

Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
Coordinates 51.212650299072266°N 4.370503902435303°E
Elevation 5.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


In [119]:
rdf

,date,temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
10995,2017-12-26 15:49:00+00:00,6.195500,80.821373,3.1455,1.244465,0.0,0.0,0.0,0.0,1.0,...,34.0,249.075058,54.000000,53.881691,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
83440,2017-12-22 11:32:00+00:00,9.695499,94.122681,8.7955,7.522425,0.0,0.0,0.0,0.0,3.0,...,38.0,3.920862,39.000000,360.506653,40.455433,1.037319,39.418114,3.920862,40.455433,373.960358
68762,2017-12-01 09:24:00+00:00,-0.004500,91.961853,-1.1545,-4.277633,0.0,0.0,0.0,0.0,3.0,...,27.0,0.000000,27.000000,222.465897,34.787827,0.000000,34.787827,0.000000,34.787827,286.633514
82594,2018-01-11 15:52:00+00:00,8.195499,89.956543,6.6455,6.380136,0.1,0.1,0.0,0.0,51.0,...,42.0,186.149979,59.999992,86.833237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
21612,2018-01-06 10:19:00+00:00,4.945500,92.918343,3.8955,3.219722,0.0,0.0,0.0,0.0,3.0,...,48.0,0.000000,48.000000,286.888397,56.213364,0.000000,56.213364,0.000000,56.213364,335.978363
111479,2018-02-04 01:43:00+00:00,0.995500,97.159271,0.5955,-3.413191,0.0,0.0,0.0,0.0,3.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
109827,2018-01-12 19:04:00+00:00,5.495500,95.248558,4.7955,3.178516,0.0,0.0,0.0,0.0,3.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16017,2018-01-30 08:08:00+00:00,1.245500,97.515434,0.8955,-2.037434,0.0,0.0,0.0,0.0,0.0,...,5.0,49.327099,8.999997,54.059608,20.997818,9.332363,11.665454,104.312057,9.929446,126.125977
14500,2018-01-16 07:12:00+00:00,5.395500,87.248756,3.4455,-0.493744,0.4,0.4,0.0,0.0,51.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
48216,2018-01-11 09:22:00+00:00,3.795500,96.538887,3.2955,1.586693,0.0,0.0,0.0,0.0,1.0,...,27.0,119.048210,42.000008,175.361481,59.250771,21.160990,38.089783,120.969048,58.309956,247.388168


In [120]:
sample_ds


,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,BS 65,BS 66,BS 67,BS 68,RX Time,SF,HDOP,Latitude,Longitude,Weather param sense time
10995,-104,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2017-12-26 15:49:28.285000+00:00,9,1.27,51.215576,4.408385,2017-12-26T15:49
83440,-200,-200,-200,-101,-200,-200,-200,-200,-109,-200,...,-200,-200,-200,-200,2017-12-22 11:32:49.307000+00:00,7,0.61,51.235497,4.420784,2017-12-22T11:32
68762,-104,-99,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2017-12-01 09:24:43.480000+00:00,12,0.65,51.199692,4.411455,2017-12-01T09:24
82594,-200,-99,-200,-200,-200,-200,-200,-200,-200,-99,...,-200,-200,-200,-200,2018-01-11 15:52:19.204000+00:00,12,0.71,51.194435,4.418565,2018-01-11T15:52
21612,-200,-116,-200,-200,-200,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2018-01-06 10:19:29.931000+00:00,12,0.67,51.207184,4.410908,2018-01-06T10:19
111479,-200,-200,-200,-200,-105,-200,-200,-200,-200,-200,...,-200,-200,-200,-200,2018-02-04 01:43:13.705000+00:00,8,1.14,51.235714,4.415915,2018-02-04T01:43
109827,-103,-94,-200,-200,-200,-200,-200,-200,-200,-84,...,-200,-200,-200,-200,2018-01-12 19:04:52.363000+00:00,12,0.70,51.194229,4.418320,2018-01-12T19:04
16017,-200,-200,-200,-200,-200,-200,-200,-200,-98,-200,...,-200,-200,-200,-200,2018-01-30 08:08:04.400000+00:00,7,0.60,51.215614,4.420460,2018-01-30T08:08
14500,-200,-200,-200,-200,-200,-200,-200,-200,-104,-200,...,-200,-200,-200,-200,2018-01-16 07:12:26.641000+00:00,7,0.61,51.214912,4.437533,2018-01-16T07:12
48216,-200,-200,-200,-200,-200,-200,-121,-200,-111,-200,...,-200,-200,-200,-200,2018-01-11 09:22:19.133000+00:00,8,0.62,51.229065,4.442570,2018-01-11T09:22


In [82]:

try:
    rdf = ds.sample(n=50).apply(get_weather_info_mod, axis=1)
except Exception as e:
    # if 'Hourly API request limit exceeded' in str(e):
    if 'Hourly' in str(e):
        print('wait')
    else:
        raise e

wait


In [121]:
ds_weather = pd.concat([sample_ds, rdf], axis=1, join='outer')

In [122]:
ds_weather

,BS 1,BS 2,BS 3,BS 4,BS 5,BS 6,BS 7,BS 8,BS 9,BS 10,...,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant
10995,-104,-200,-200,-200,-200,-200,-200,-200,-200,-200,...,34.0,249.075058,54.000000,53.881691,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
83440,-200,-200,-200,-101,-200,-200,-200,-200,-109,-200,...,38.0,3.920862,39.000000,360.506653,40.455433,1.037319,39.418114,3.920862,40.455433,373.960358
68762,-104,-99,-200,-200,-200,-200,-200,-200,-200,-200,...,27.0,0.000000,27.000000,222.465897,34.787827,0.000000,34.787827,0.000000,34.787827,286.633514
82594,-200,-99,-200,-200,-200,-200,-200,-200,-200,-99,...,42.0,186.149979,59.999992,86.833237,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
21612,-200,-116,-200,-200,-200,-200,-200,-200,-200,-200,...,48.0,0.000000,48.000000,286.888397,56.213364,0.000000,56.213364,0.000000,56.213364,335.978363
111479,-200,-200,-200,-200,-105,-200,-200,-200,-200,-200,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
109827,-103,-94,-200,-200,-200,-200,-200,-200,-200,-84,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
16017,-200,-200,-200,-200,-200,-200,-200,-200,-98,-200,...,5.0,49.327099,8.999997,54.059608,20.997818,9.332363,11.665454,104.312057,9.929446,126.125977
14500,-200,-200,-200,-200,-200,-200,-200,-200,-104,-200,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
48216,-200,-200,-200,-200,-200,-200,-121,-200,-111,-200,...,27.0,119.048210,42.000008,175.361481,59.250771,21.160990,38.089783,120.969048,58.309956,247.388168


In [123]:
ds_weather.to_csv('sample_antwerp_ds_weather-data.csv')